In [51]:
from pyrosetta.rosetta.core.pack.task import *
from pyrosetta.rosetta.protocols import *
from pyrosetta.rosetta.core.select import *

#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

import multiprocessing

#Core Includes
from rosetta.core.kinematics import MoveMap
from rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax
import multiprocessing
import pandas as pd
import numpy as np

from itertools import combinations 
from pyrosetta.toolbox import *

init("-default_max_cycles 200 -missing_density_to_jump -ex1 -ex2aro -ignore_zero_occupancy false -fa_max_dis 9 -mute all")

pose_path =  "lowest_cart_relaxed_hg3.pdb"
pose = pose_from_pdb(pose_path)

/tmp/ipykernel_50477/890472880.py:13: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta.core.kinematics import MoveMap


PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.linux 2021.45+release.b2be42856e92b441d5d806df7b34ab4d77e39eed 2021-11-10T12:38:05] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [229]:
len(pose.sequence())

318

In [55]:
pose.residue(318).name()

'HIS_D:CtermProteinFull'

In [ ]:

def mutate_repack_func4(pose, target_position, mutant, repack_radius, sfxn, ddg_bbnbrs=1, verbose=False, cartesian=True, max_iter=None):
    import time
    from pyrosetta.rosetta.core.pack.task import operation

    #logger.warning("Interface mode not implemented (should be added!)")
    
    if cartesian:
        sfxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreTypeManager.score_type_from_name('cart_bonded'), 0.5)
        #sfxn.set_weight(atom_pair_constraint, 1)#0.5
        sfxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreTypeManager.score_type_from_name('pro_close'), 0)
        #logger.warning(pyrosetta.rosetta.basic.options.get_boolean_option('ex1'))#set_boolean_option( '-ex1', True )
        #pyrosetta.rosetta.basic.options.set_boolean_option( 'ex2', True )
    
    #Cloning of the pose including all settings
    working_pose = pose.clone()

    #Select mutant residue
    mutant_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(target_position)
    
    #Select all except mutant
    all_nand_mutant_selector = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector()
    all_nand_mutant_selector.set_residue_selector(mutant_selector)

    #Select neighbors with mutant
    nbr_or_mutant_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    nbr_or_mutant_selector.set_focus(str(target_position))
    nbr_or_mutant_selector.set_distance(repack_radius)
    nbr_or_mutant_selector.set_include_focus_in_subset(True)

    #Select mutant and it's sequence neighbors
    seq_nbr_or_mutant_selector = pyrosetta.rosetta.core.select.residue_selector.PrimarySequenceNeighborhoodSelector(ddg_bbnbrs, ddg_bbnbrs, mutant_selector, False)            

    #Select mutant, it's seq neighbors and it's surrounding neighbors
    seq_nbr_or_nbr_or_mutant_selector = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector()
    seq_nbr_or_nbr_or_mutant_selector.add_residue_selector(seq_nbr_or_mutant_selector)
    seq_nbr_or_nbr_or_mutant_selector.add_residue_selector(nbr_or_mutant_selector)    

    if verbose:
        print(f'mutant_selector: {pyrosetta.rosetta.core.select.residue_selector.selection_positions(mutant_selector.apply(working_pose))}')
        print(f'all_nand_mutant_selector: {pyrosetta.rosetta.core.select.residue_selector.selection_positions(all_nand_mutant_selector.apply(working_pose))}')
        print(f'nbr_or_mutant_selector: {pyrosetta.rosetta.core.select.residue_selector.selection_positions(nbr_or_mutant_selector.apply(working_pose))}')
        print(f'seq_nbr_or_mutant_selector: {pyrosetta.rosetta.core.select.residue_selector.selection_positions(seq_nbr_or_mutant_selector.apply(working_pose))}')
        print(f'seq_nbr_or_nbr_or_mutant_selector: {pyrosetta.rosetta.core.select.residue_selector.selection_positions(seq_nbr_or_nbr_or_mutant_selector.apply(working_pose))}')
     
    
    #Mutate residue and pack rotamers before relax
    #if list(pose.sequence())[target_position-1] != mutant:
        #generate packer task
    tf = TaskFactory()
    tf.push_back(operation.InitializeFromCommandline())
    tf.push_back(operation.IncludeCurrent())

    #Set all residues except mutant to false for design and repacking
    prevent_repacking_rlt = operation.PreventRepackingRLT()
    prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, all_nand_mutant_selector, False )
    tf.push_back(prevent_subset_repacking)

    #Assign mutant residue to be designed and repacked
    resfile_comm = pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation(mutant_selector, f"PIKAA {mutant}")
    resfile_comm.set_command(f"PIKAA {mutant}")
    tf.push_back(resfile_comm)

    #Apply packing of rotamers of mutant
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.score_function(sfxn)
    packer.task_factory(tf)
    if verbose:
        logger.warning(tf.create_task_and_apply_taskoperations(working_pose))
    packer.apply(working_pose)
        
    #allow the movement for bb for the mutant + seq. neighbors, and sc for neigbor in range, seq. neighbor and mutant
    movemap = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    movemap.all_jumps(False)
    movemap.add_bb_action(pyrosetta.rosetta.core.select.movemap.mm_enable, seq_nbr_or_mutant_selector)
    movemap.add_chi_action(pyrosetta.rosetta.core.select.movemap.mm_enable, seq_nbr_or_nbr_or_mutant_selector)
    
    #for checking if all has been selected correctly
    #if verbose:
    mm  = movemap.create_movemap_from_pose(working_pose)
    
    logger.info(mm)

    #Generate a TaskFactory
    tf = TaskFactory()
    tf.push_back(operation.InitializeFromCommandline())
    tf.push_back(operation.IncludeCurrent())
    #tf.push_back(operation.NoRepackDisulfides())

    #prevent all residues except selected from design and repacking
    prevent_repacking_rlt = operation.PreventRepackingRLT()
    prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, seq_nbr_or_nbr_or_mutant_selector, True )
    tf.push_back(prevent_subset_repacking)

    # allow selected residues only repacking (=switch off design)
    restrict_repacking_rlt = operation.RestrictToRepackingRLT()
    restrict_subset_repacking = operation.OperateOnResidueSubset(restrict_repacking_rlt , seq_nbr_or_nbr_or_mutant_selector, False)
    tf.push_back(restrict_subset_repacking)


    #Perform a FastRelax
    fastrelax = pyrosetta.rosetta.protocols.relax.FastRelax()
    fastrelax.set_scorefxn(sfxn)
    
    if cartesian:
        fastrelax.cartesian(True)
    if max_iter:
        fastrelax.max_iter(max_iter)
        
    fastrelax.set_task_factory(tf)
    fastrelax.set_movemap_factory(movemap)
    fastrelax.set_movemap_disables_packing_of_fixed_chi_positions(True)
    
    if verbose:
        logger.info(tf.create_task_and_apply_taskoperations(working_pose))
    fastrelax.apply(working_pose)
    return working_pose

def cart_ddg(site,res):
    
    newpose = pose.clone()
    scores = []
    
    for i in range(3):
        scorefxn = create_score_function("ref2015_cart")
        newpose = mutate_repack_func4(newpose,site, res, 6, scorefxn,verbose = False, cartesian = True)
        news = scorefxn(newpose)
        scores.append(news)
    return scores

all_as = sorted(list(set(pose.sequence())))
sites = np.arange(1,len(pose.sequence()) + 1)
assert len(all_as) == 20

inputs_ = [[site,res] for site in sites for res in all_as]
print(len(inputs_))
cores = os.cpu_count()
print(cores)

with multiprocessing.Pool(processes=cores) as pool:
    results = pool.starmap(cart_ddg,inputs_)

import pickle
with open('hg3_base.pkl', 'wb') as f:
    pickle.dump(results, f)

6360
64


In [ ]:
print("HI")

In [59]:
with open('hg3_base_inputs_.pkl', 'wb') as f:
    pickle.dump(inputs_, f)

In [71]:
results_df = pd.concat([pd.DataFrame(results).add_prefix("iter_"), pd.DataFrame(inputs_).rename(columns = {0:"site", 1:"AS"})], axis=1)

In [228]:
len(results)

6360

In [81]:
newl = [[x] * 20 for x in list(pose.sequence())]

In [85]:
newl = [i for b in newl for i in b]

In [86]:
results_df["wt"] = newl

In [94]:
results_df.loc[results_df.site == 262]

,iter_0,iter_1,iter_2,site,AS,wt
5220,-1633.625488,-1632.821125,-1632.809370,262,A,S
5221,-1633.031263,-1633.009751,-1633.041426,262,C,S
5222,-1630.155572,-1629.826864,-1629.407498,262,D,S
5223,-1632.264704,-1630.984686,-1631.047056,262,E,S
5224,-1638.384636,-1637.384203,-1637.451840,262,F,S
5225,-1628.437889,-1627.284049,-1627.349413,262,G,S
5226,-1636.149473,-1634.971272,-1635.084748,262,H,S
5227,-1634.273550,-1633.532572,-1633.510524,262,I,S
5228,-1634.301202,-1633.594705,-1633.368081,262,K,S
5229,-1636.408688,-1635.661469,-1635.605673,262,L,S


In [105]:
results_df["e_min"] = results_df.apply(lambda x: x[["iter_0", "iter_1", "iter_2"]].min(), axis=1)

In [126]:
def ddg_df(sub_df):
    
    wt_min = sub_df.loc[sub_df["AS"] == sub_df["wt"], "e_min"].values[0]
    return sub_df["e_min"].astype(float) - wt_min

In [137]:
ddg_df(results_df.loc[results_df.site == 269])

5360   -3.940947
5361    6.286431
5362    6.808921
5363    6.314160
5364    0.000000
5365    2.412121
5366    3.967802
5367    2.786256
5368    5.670689
5369    0.540267
5370   -0.387767
5371    4.211200
5372    1.833857
5373    4.275715
5374    3.817930
5375    7.788693
5376    7.502538
5377    1.020159
5378    1.403104
5379    0.069857
Name: e_min, dtype: float64

In [134]:
results_df["ddg"] = results_df.groupby("site").apply(ddg_df).reset_index(drop=True)

In [136]:
results_df.sort_values("ddg")

,iter_0,iter_1,iter_2,site,AS,wt,e_min,ddg
3119,-1645.980957,-1645.815413,-1645.857387,156,Y,P,-1645.980957,-14.330316
3104,-1645.573865,-1645.119329,-1644.970302,156,F,P,-1645.573865,-13.923224
3118,-1645.463928,-1644.439981,-1644.516454,156,W,P,-1645.463928,-13.813288
4818,-1644.064647,-1644.968821,-1644.356004,241,W,D,-1644.968821,-12.464130
3117,-1643.774205,-1642.911256,-1643.067028,156,V,P,-1643.774205,-12.123565
...,...,...,...,...,...,...,...,...
1698,-1590.807130,-1590.982154,-1590.972971,85,W,G,-1590.982154,40.750548
1686,-1590.241100,-1590.361308,-1590.440508,85,H,G,-1590.440508,41.292195
1694,-1588.214729,-1588.443151,-1588.525754,85,R,G,-1588.525754,43.206948
312,-1576.636048,-1575.796009,-1576.653745,16,P,G,-1576.653745,54.647775


In [151]:
results_df.loc[results_df.site == 156].sort_values("ddg").reset_index(drop=True)

,iter_0,iter_1,iter_2,site,AS,wt,e_min,ddg
0,-1645.980957,-1645.815413,-1645.857387,156,Y,P,-1645.980957,-14.330316
1,-1645.573865,-1645.119329,-1644.970302,156,F,P,-1645.573865,-13.923224
2,-1645.463928,-1644.439981,-1644.516454,156,W,P,-1645.463928,-13.813288
3,-1643.774205,-1642.911256,-1643.067028,156,V,P,-1643.774205,-12.123565
4,-1643.555324,-1642.734794,-1642.640894,156,I,P,-1643.555324,-11.904684
5,-1643.306802,-1642.118814,-1642.155368,156,R,P,-1643.306802,-11.656161
6,-1643.236150,-1642.424428,-1642.457792,156,L,P,-1643.236150,-11.585509
7,-1642.973921,-1642.749398,-1642.877401,156,M,P,-1642.973921,-11.323281
8,-1642.147117,-1641.085781,-1641.260718,156,A,P,-1642.147117,-10.496476
9,-1641.633878,-1641.362622,-1641.434132,156,H,P,-1641.633878,-9.983238


In [198]:
prob_df = pd.read_csv("prob_df.csv")

prob_df = prob_df.div(prob_df.sum(axis=1), axis=0).fillna(0)

prob_df = prob_df.reset_index() 
prob_df.rename(columns = {"index":"site"}, inplace=True)
prob_df["site"] += 1

prob_list = []

for r,row in prob_df.iterrows():
    prob_list.append([(row["site"],b,a) for a,b in zip(row[1:], row[1:].index)])
    
prob_list = [i for b in prob_list for i in b]

prob_df = pd.DataFrame(prob_list, columns = ["site","AS", "p"])

prob_df["id"] = prob_df.apply(lambda x: x["AS"] + str(int(x["site"])), 1)

results_df["id"] = results_df.apply(lambda x: x["AS"] + str(int(x["site"])), 1)

newr = pd.merge(results_df,prob_df[["id","p"]], how = "left", on = "id")

newr.dropna()[["ddg","p"]].corr()

In [208]:
newr.loc[newr.ddg < -1][["ddg","p"]].corr()

,ddg,p
ddg,1.000000,0.011251
p,0.011251,1.000000


In [213]:
newr.loc[(newr.ddg < -1) & (newr.p != 0)].dropna()[["ddg","p"]].corr()

,ddg,p
ddg,1.00000,0.00769
p,0.00769,1.00000


In [216]:
newr.loc[(newr.ddg < -1) & (newr.p != 0)].dropna()["p"].mean()

0.05174984608216648

In [217]:
newr.loc[(newr.ddg > -1) & (newr.p != 0)].dropna()["p"].mean()

0.07046363107011272

In [201]:
newr.loc[newr.p != 0].dropna()[["ddg","p"]].corr()

,ddg,p
ddg,1.000000,-0.155168
p,-0.155168,1.000000


In [227]:
results_df.loc[results_df.site == 76]

,iter_0,iter_1,iter_2,site,AS,wt,e_min,ddg,id
1500,-1630.394373,-1629.544735,-1630.568720,76,A,Q,-1630.568720,1.074586,A76
1501,-1630.527800,-1629.791971,-1629.943036,76,C,Q,-1630.527800,1.115506,C76
1502,-1632.146717,-1630.938347,-1630.936715,76,D,Q,-1632.146717,-0.503411,D76
1503,-1632.372015,-1631.421906,-1631.247713,76,E,Q,-1632.372015,-0.728709,E76
1504,-1633.152140,-1632.238954,-1632.130577,76,F,Q,-1633.152140,-1.508834,F76
1505,-1626.975147,-1626.093076,-1626.100571,76,G,Q,-1626.975147,4.668159,G76
1506,-1630.844112,-1629.920389,-1629.819821,76,H,Q,-1630.844112,0.799193,H76
1507,-1632.948727,-1631.726046,-1631.751137,76,I,Q,-1632.948727,-1.305421,I76
1508,-1629.880898,-1628.994579,-1629.015581,76,K,Q,-1629.880898,1.762407,K76
1509,-1634.321793,-1633.242258,-1634.128332,76,L,Q,-1634.321793,-2.678487,L76
